<a href="https://colab.research.google.com/github/AchillesHackerGod/Secret-Santa-with-Pyhton/blob/main/SecretSanta_(1).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>



 **Adjustments still open for the code:**

- Make the email looks better
- Make sure the code is understandable and easy to run.
- Does the reading the URL allways work?


In [124]:
import pandas as pd
import random as rd
import numpy as np
import smtplib
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText


In [125]:
!pip install openai==0.28
import openai

**In case you run the code in google colab:**
1.The sk.py file must be stored in your drive.
2. Link the google drive with colab. Make sure the path is right.
3. Adjust the path if necassary and run the code right below

In [126]:
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/MyDrive/Colab Notebooks
%ls

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/MyDrive/Colab Notebooks
'Kopie von SecretSanta (1).ipynb'  'SecretSanta (1).ipynb'   Untitled0.ipynb
 __pycache__/                       sk1.py


In [132]:
from sk1 import api_acc1, mail_acc
openai.api_key = api_acc1

In [128]:
url = 'https://docs.google.com/spreadsheets/d/1IAOMmHcYTbVPPxj3Dk65r97R76bLVXWHdL4W7wh6ff0/edit#gid=1191019850'
url_1 = url.replace('/edit#gid=', '/export?format=csv&gid=')

In [129]:
# The google survey is set to automatically write a google docs document that includes all the information from the survey
# Google docs generates a csv file that is read into a DataFrame
df = pd.read_csv(url_1)
df

,Timestamp,Name,E-Mail,What is your wish from secret santa?,Choose or add a theme for your gift.
0,08/11/2023 16:44:50,Achilleas,a.stemshorn@gmail.com,Playstation 5,Fortnite
1,09/11/2023 13:19:51,I hui,b11703058@ntu.edu.tw,Chocolate,Romance
2,22/11/2023 14:50:47,Leonard M-M,l.meyer-marc@web.de,Nintendo Color,Sustainability
3,22/11/2023 15:04:36,Fede,federicaballardini@outlook.com,"world peace, more and tinier kittens, the ulti...",kittens


In [130]:
mail = df['E-Mail'].tolist()
names = df['Name'].tolist()

In [133]:
# Open up a new data frame for the Secret Santa and reciever with all necessary information
df_out = pd.DataFrame()
# Copy the names list in a new list which will be modified later
secret_santa_list = names.copy()

stopper = 0 #loop stopper
recipient = []

# In two cases the code must be considered
    # 1 When Secret Santa  = Recipient the randit function must be run again
    # 2 When for the last draw (one name left as a potential recipient) the case Santa = Reciepient is true the hole code must run again
    #otherwise we have an infinite loop

while(stopper == 0):

    redo = False

    secret_santa_list = names.copy()

    for i in range(0, len(names)):
        recip = rd.randint(0, len(secret_santa_list) - 1)
        x = 0
        while(x == 0):
            if(names[i] == secret_santa_list[recip]):
                if(len(secret_santa_list) == 1):
                    redo = True
                    x = 1
                else:
                    recip = rd.randint(0, len(secret_santa_list) - 1)
            else:
                x = 1


        if(redo != True):
            recipient.append(secret_santa_list[recip])
            secret_santa_list.pop(recip)
            stopper = 1
        else:
            stopper = 0
 # make the new data frame with all necessary inforamtion
for w,k in zip(wish_list,np.arange(len(names))):
  df_out.loc[k,'Secret Santa'] = names[k]
  df_out.loc[k,'Recipient'] = recipient[k]

  index_values = df[df['Name'] == recipient[k]].index.values
  df_out.loc[k,'E-Mail'] = df['E-Mail'].iloc[index_values[0]]
  df_out.loc[k, 'Wish'] = df['What is your wish from secret santa?'].iloc[index_values[0]]
  df_out.loc[k,'Theme'] = df['Choose or add a theme for your gift.'].iloc[index_values[0]]

  # getting the recommendations from openai and adding the to the DataFrame

  recommendations = openai.ChatCompletion.create(model="gpt-3.5-turbo",
                                messages=[{"role": "user", "content": f"List me 3 sustainable sectet santa gift recommandations which are related to {w}."}],
                                max_tokens = 200, n=1)
  df_out.loc[k,'Recommendations'] = recommendations['choices'][0]['message']['content']

#easiest way to access the information is in form of new lists, the data frame can be used to have a better overview of all partners in case something goes sideways in the emailing part
recommendations_list = df_out['Recommendations'].tolist()
recipient_list = df_out['Recipient'].tolist()
wish_list = df_out['Wish'].tolist()
theme_list = df_out['Theme'].tolist()
df_out

,Secret Santa,Recipient,E-Mail,Wish,Theme,Recommendations
0,Achilleas,I hui,b11703058@ntu.edu.tw,Chocolate,Romance,1. Nintendo Eco-Friendly Switch Case: A sustai...
1,I hui,Leonard M-M,l.meyer-marc@web.de,Nintendo Color,Sustainability,1. World Peace Candle Set: This set includes s...
2,Leonard M-M,Achilleas,a.stemshorn@gmail.com,Playstation 5,Fortnite,1. Eco-friendly Chocolate Bars: Look for choco...
3,Fede,Fede,federicaballardini@outlook.com,"world peace, more and tinier kittens, the ulti...",kittens,1. Energy-saving gaming peripherals: Look for ...


In [134]:
sender_address = 'secret.santa241200@gmail.com'
sender_pass = mail_acc
for i in range(0, len(names)):

    # Gets the reciever mail from the list which was produced by the code above
    receiver_address = mail[i]

    # sets up the MIME
    message = MIMEMultipart()
    message['From'] = sender_address
    message['To'] = receiver_address
    message['Subject'] = 'Secret Santa'

    # You can replace the the image if you want. The source mus be a URL link like 'https://example.com/christmas_image.jpg'
    # A website that can be used is https://wallpapercave.com/search?q=christmas. With right click on the image you can choose "copy image link"
    html = f'''
    <html>
      <head>
        <style>
          body {{
            font-family: 'Arial', sans-serif;
            background-color: #f0f0f0;
            margin: 0;
            padding: 0;
          }}

          .container {{
            max-width: 600px;
            margin: 0 auto;
            background-color: #fff;
            padding: 20px;
            border-radius: 10px;
            box-shadow: 0 0 10px rgba(0, 0, 0, 0.1);
          }}

          h1 {{
            color: #d9534f;
            text-align: center;
          }}

          p {{
            font-size: 16px;
            line-height: 1.6;
            color: #333;
          }}

          p strong {{
            font-weight: bold;
          }}

          img {{
            display: block;
            margin: 20px auto;
            width: 100%; /* Adjust the width as needed */
          }}
        </style>
      </head>
      <body>
        <div class="container">
          <h1>Merry Christmas!</h1>
          <img src="https://wallpapercave.com/uwp/uwp4148641.jpeg" alt="Christmas Image" width="50%">
          <p>Dear {names[i]},</p>
          <p>Wishing you a joyous and magical Christmas filled with love, laughter, and warmth. May the holiday season bring you happiness and fond memories with your loved ones.</p>
          <p>You are the Secret Santa of <strong>{recipient_list[i]}</strong>. He/She wishes a <strong>{wish_list[i]}</strong> </p>
          <p> Sustainable gift recommendations related to <strong>{wish_list[i]}</strong> by OpenAI are: </p>
          <p> {recommendations_list[i]} </p>
          <p>Warm regards,</p>
          <p>Your sustainable Secret Santa Python Program</p>
        </div>
      </body>
    </html>
    '''
    # Attach html to the mail
    message.attach(MIMEText(html, 'html'))

    # Creates the SMTP session for sending the mail
    session = smtplib.SMTP(host='smtp.gmail.com', port=587)
    session.starttls()
    session.login(sender_address, sender_pass)
    text = message.as_string()
    session.sendmail(sender_address, receiver_address, text)
    session.quit()
